<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Align,-combine,-subset-EPA-radon,-uranium-data-and-US-Census-GIS-data" data-toc-modified-id="Align,-combine,-subset-EPA-radon,-uranium-data-and-US-Census-GIS-data-1">Align, combine, subset EPA radon, uranium data and US Census GIS data</a></span><ul class="toc-item"><li><span><a href="#Notebook-setup" data-toc-modified-id="Notebook-setup-1.1">Notebook setup</a></span></li><li><span><a href="#State/EPA-Residiental-Survey-(SRRS)-datasets" data-toc-modified-id="State/EPA-Residiental-Survey-(SRRS)-datasets-1.2">State/EPA Residiental Survey (SRRS) datasets</a></span></li><li><span><a href="#US-census-county-boundaries-GIS-files" data-toc-modified-id="US-census-county-boundaries-GIS-files-1.3">US census county boundaries GIS files</a></span></li><li><span><a href="#EPA/State-Residential-Radon-Data" data-toc-modified-id="EPA/State-Residential-Radon-Data-1.4">EPA/State Residential Radon Data</a></span></li><li><span><a href="#US-county-soil-uranium-levels" data-toc-modified-id="US-county-soil-uranium-levels-1.5">US county soil uranium levels</a></span></li><li><span><a href="#Join-and-merge-tables-using-US-FIPS-codes" data-toc-modified-id="Join-and-merge-tables-using-US-FIPS-codes-1.6">Join and merge tables using US FIPS codes</a></span></li><li><span><a href="#County-level-information:---uranium,-number-of-homes-in-radon-survey,-census-county-name" data-toc-modified-id="County-level-information:---uranium,-number-of-homes-in-radon-survey,-census-county-name-1.7">County level information:   uranium, number of homes in radon survey, census county name</a></span></li><li><span><a href="#Restrict-dataset-to-Minnesota" data-toc-modified-id="Restrict-dataset-to-Minnesota-1.8">Restrict dataset to Minnesota</a></span></li></ul></li></ul></div>

## Align, combine, subset EPA radon, uranium data and US Census GIS data

### Notebook setup

In [ ]:
# import all libraries used in this notebook
import os
import numpy as np
import pandas as pd
import geopandas as gpd

### State/EPA Residiental Survey (SRRS) datasets

The rawest form of the radon data was collected and archived by Phil Price and is available here:
http://www.stat.columbia.edu/~gelman/arm/examples/radon_complete


* The documentation is in file http://www.stat.columbia.edu/~gelman/arm/examples/radon_complete/SRRSdoc.pdf

* There are 5 files, srrs1.dat through srrs5.dat  - but data is duplicated between them.

* This directory also contains data from both national survey - NRRS - and state surveys - cf. https://link.springer.com/article/10.1007/BF02034901.   This is in a different format and is
not used in Gelman and Hill analysis.

* README notes that files are old backups, things may be missing.

The combined de-duplicated SRRS dataset is in file  [srrs_all.csv](data/srrs_all.csv)

*State counties and tribal lands*

The SRRS dataset contains observations taken from Indian lands.
The county-level information for these entries doesn't line up with US FIPS data -
the names and county codes don't align.
Indian lands have column 'STATE' code R5, R6, R7, RB, RC, RN.
The regions cross state boundaries - for example,
EPA region 5 covers Indian lands in MN, WI, and MI:
https://www.epa.gov/sites/default/files/2015-08/documents/r5-tribal-land-map.pdf.

Data in state counties in file [radon_all_states.csv](data/radon_all_states.csv).

Data from Indian lands is in file [radon_indg_lands.csv](data/radon_indg.csv).


### US census county boundaries GIS files

The US Census provides shapefiles for the US, including Alaska, Hawaii, and territories.  We can use these to visualize radon and uranium levels.

In [ ]:
shpfile = os.path.join('geo_data','cb_2018_us_county_20m', 'cb_2018_us_county_20m.shp')
us_geodata = gpd.read_file(shpfile)
# GEOID should be numeric
us_geodata = us_geodata.astype({'GEOID': 'int32'}, copy=False)
print(us_geodata.shape[0])
us_geodata.head(3)

### EPA/State Residential Radon Data

In [ ]:
us_radon = pd.read_csv(os.path.join('data','radon_all_counties.csv'),
                     usecols=['state', 'stfips', 'floor', 'activity', 'cntyfips'],
                     skipinitialspace=True,    # CSV file has spaces after delimiter, ignore them
    ).convert_dtypes()
print(us_radon.shape)
us_radon.head(3)

**datacleanup**

Colorado and CT have data with cntyfips codes '0' and '999'.   Dropping for now.

In [ ]:
us_radon.drop(us_radon[us_radon.cntyfips==0].index, inplace=True)
us_radon.drop(us_radon[us_radon.cntyfips==999].index, inplace=True)
print(us_radon.shape)

### US county soil uranium levels

Also distributed from Gelman website.

In [ ]:
us_uranium = pd.read_csv(os.path.join('data','raw_uranium.csv'),
                        usecols=['st', 'stfips', 'ctfips', 'Uppm'],
                        skipinitialspace=True,
                        ).drop_duplicates().convert_dtypes()
print(us_uranium.shape[0])
us_uranium.head(3)

### Join and merge tables using US FIPS codes

To join or merge tables, we need to create a common key in both, then
use the [DataFrame.merge](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html) method.


We have three datasets:  SRRS survey data, soil uranium measurements, and geodata.
All files use different capitalization and punctuation for county names.
Therefore we rely on 
[FIPS code](https://transition.fcc.gov/oet/info/maps/census/fips/fips.txt),
which uniquely identify geographic areas. 
The US census datasets have "GEOID" code, the first 2 digits of which are the state FIPS code, the last 3 are the county-level FIPS code.
The other datasets have separate columns for stats and county codes.

In [ ]:
# create merge column
us_uranium['FIPS'] = us_uranium.stfips*1000 + us_uranium.ctfips
us_radon['FIPS'] = us_radon.stfips*1000 + us_radon.cntyfips

### County level information:   uranium, number of homes in radon survey, census county name

We create a new table which contains county-level information from across the three datasets.

In [ ]:
us_counties = us_uranium.merge(us_geodata[['GEOID', 'NAME']],
                               how='inner', left_on='FIPS', right_on='GEOID')

homes = us_radon.value_counts(subset=['FIPS'], sort=False).to_frame().reset_index()
homes.rename(columns={0:'homes'}, inplace=True)

us_counties = us_counties.merge(homes, how='left', on='FIPS')
us_counties.fillna(0, inplace=True)

us_counties.drop(columns=['stfips', 'ctfips', 'GEOID'], inplace=True)
us_counties.rename(columns={'st': 'state', 'NAME':'county', 'Uppm':'uranium'}, inplace=True)

print(us_counties.shape[0])
us_counties.head(3)

In [ ]:
us_counties[us_counties.state=='MN'].shape

#### Put data on log scale

Following Gelman and Hill chapter 4, section 4, we work with data on the log scale,
for two reasons

+ the outcome variable log_radon is always positive.
+ it provides modeling flexibility.

We know from geology that both radon measurements and soil uranium levels are always greater than zero,
however a few radon measurements in the EPA dataset are 0.
In order to be able to work with these measurements on the log scale, we replace 0 with 0.1,
which corresponds to a low radon level (following Gelman and Hill).

In [ ]:
us_radon['radon'] = us_radon.activity.apply(lambda x: x if x > 0.1 else 0.1)
us_radon['log_radon'] = np.log(us_radon['radon'])
us_radon.drop(columns=['activity', 'stfips', 'cntyfips'], inplace=True)
us_radon.head(3)

In [ ]:
us_counties.uranium.fillna(0.1, inplace=True)
us_counties['u'] = us_counties.uranium.apply(lambda x: x if x > 0.1 else 0.1)
us_counties['log_uranium'] = np.log(us_counties['u'])
us_counties.drop(columns=['u'], inplace=True)
us_counties.head(3)

### Restrict dataset to Minnesota

In order to work with just the data from Minnesota, we use a 
use a conditional expression to [filter specific rows of a dataframe](https://pandas.pydata.org/docs/getting_started/intro_tutorials/03_subset_data.html#how-do-i-filter-specific-rows-from-a-dataframe), combined with operation [reset_index(drop=True)](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.reset_index.html?highlight=reset_index#pandas.DataFrame.reset_index) so that the rows are indexed starting from 0.

In [ ]:
mn_radon = us_radon[us_radon['state']=='MN'].reset_index(drop=True)
mn_radon.drop(columns=['state'], inplace=True)
mn_radon = mn_radon.merge(us_counties[['FIPS', 'county']], on='FIPS')
mn_radon = mn_radon.sort_values(by='county', axis=0).reset_index(drop=True)
mn_radon.head(3)

In [ ]:
mn_counties = us_counties[us_counties['state']=='MN'].reset_index(drop=True)
mn_counties.drop(columns=['state'], inplace=True)
mn_counties.head(3)

#### Unique county ids

In [ ]:
# super clunky to index offset from 1
mn_counties.reset_index(inplace=True)
mn_counties['county_id'] = mn_counties.index + 1
mn_counties.drop(columns=['index'], inplace=True)
mn_counties.head(3)

Add county ids to radon data as well.

In [ ]:
mn_radon = mn_radon.merge(mn_counties[['FIPS', 'county_id']], on='FIPS')
mn_radon.head(3)

**Save as CSV files**

These files are already part of this notebook, therefore calls to the  [pandas.to_csv](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_csv.html?highlight=to_csv#pandas.DataFrame.to_csv) method have been commented out.

In [ ]:
# uncomment as needed
# mn_radon.to_csv(os.path.join('data', 'mn_radon.csv'), index=False)
# mn_counties.to_csv(os.path.join('data', 'mn_counties.csv'), index=False)

# us_radon.to_csv(os.path.join('data', 'us_radon.csv'), index=False)
# us_counties.to_csv(os.path.join('data', 'us_counties.csv'), index=False)